# Authentiacate GEE

In [1]:
# This will track the datasets version if any change is made to the code
GEE_version = 1

In [2]:
import ee
# Trigger the authentication flow for Google Earth Engine (GEE).
ee.Authenticate()
# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AQlEd8ymL_JdzJqlRXhDJNDKM_HUhlPZtFniEYmpw-V8mi23Tqm4yMMtScM



Successfully saved authorization token.


In [3]:
# Loading all the nessesary packages
import datetime
import geetools
import geemap
import concurrent.futures
from tqdm import tqdm
import pandas as pd
import numpy as np

In [4]:
# Load the feature collection (i.e., boundary shapefile in csv format) needed to simulate aggregation
# Note: that this csv file will just contain countries names, but the actual boundary (be it first or second level admin boundary) that is being simulated should be uploaded in GEE
GADM_countries = pd.read_csv("/home/chandra/backup/Chalmers/GADM_Boundaries.csv")

# Groupby and count the attributes/shapes in the dataframe, just to get a sense of number of distinct sub-boundaries within a country
counts = GADM_countries.groupby('COUNTRY').size().reset_index(name='count')

# Resulting dataframe with distinct Countries and unique sub-boundaries within them
Countries = pd.DataFrame(counts[['COUNTRY', 'count']]).sort_values(by='count')

# Check total countries that will be simulated
GADM_countries['COUNTRY'].unique().shape

(253,)

In [5]:
def GEE_loop(Admin_boundary, simulation = 'All'):
    global geometry, uniqueValues_ADM2CODE
    
    ####################
    # 1. Intial run values 
    ####################
    
    # Add end year , i.e., the year utpto which the attribution simulation runs
    end_year = 2022
    # Assuming tree cover greater than and equal to (≥) 25% is considered forest in Hansen dataset
    Forest_threshold = 25
    # Forestloss attribution to plantation, by assuming that pixels with startyear >2000 are due to deforestation after year 2000, the rest is considered under deforestation pre-2000's
    Plantation_threshold_year = 2000
    
    
    
    
    ####################
    # 2. Call/Check latest datasets 
    ####################
    
    # Load the feature collection (i.e., shapefile) needed to simulate aggregation 
    GADM_admin_boundary_version = "projects/lu-chandrakant/assets/GADM_dissolved"   # This file is downloaded from GADM (version 4.1; https://gadm.org/data.html) and uploaded to google earth engine
    
    # Other raster datasets
    Hansen_tree_cover_dataset_version = "UMD/hansen/global_forest_change_2022_v1_10" # Check the latest version of Hansen et al. 2013 (https://www.science.org/doi/10.1126/science.1244693p) at https://glad.earthengine.app/view/global-forest-change
    Plantation_dataset_version = "projects/lu-chandrakant/assets/Plantation_new/"    # Check latest plantation dataset from Du et al. 2022 (https://www.nature.com/articles/s41597-022-01260-2) and upload to GEE
    
    # Update/Check MapBiomas dataset colllection (Integrated maps)
    MapBiomas_dataset_version = {'Amazon': 'projects/mapbiomas-raisg/public/collection5/mapbiomas_raisg_panamazonia_collection5_integration_v1',                  #MapBiomas-Amazonia (1985-2022) 
                                  'Argentina': 'projects/mapbiomas-public/assets/argentina/collection1/mapbiomas_argentina_collection1_integration_v1',           #MapBiomas-Argentina (1998-2022)
                                  'Atlantic forest': 'projects/mapbiomas_af_trinacional/public/collection3/mapbiomas_atlantic_forest_collection30_integration_v1',#MapBiomas-Atlantic forest (1985-2022)
                                  'Brazil': 'projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1',                              #MapBiomas-Brazil (1985-2022)
                                  'Chaco': 'projects/mapbiomas-chaco/public/collection4/mapbiomas_chaco_collection4_integration_v1',                              #MapBiomas-Chaco (1985-2022)
                                  'Chile': 'projects/mapbiomas-public/assets/chile/collection1/mapbiomas_chile_collection1_integration_v1',                       #MapBiomas-Chile (2000-2022)
                                  'Colombia': 'projects/mapbiomas-public/assets/colombia/collection1/mapbiomas_colombia_collection1_integration_v1',              #MapBiomas-Colombia (1985-2022)
                                  'Ecaudor': 'projects/mapbiomas-public/assets/ecuador/collection1/mapbiomas_ecuador_collection1_integration_v1',                 #MapBiomas-Ecaudor (1985-2022)
                                  'Indonesia': 'projects/mapbiomas-indonesia/public/collection2/mapbiomas_indonesia_collection2_integration_v1',                  #MapBiomas-Indonesia (2000-2022)
                                  'Pampa': 'projects/MapBiomas_Pampa/public/collection3/mapbiomas_pampa_collection3_integration_v1',                              #MapBiomas-Pampa (1985-2022)
                                  'Paraguay': 'projects/mapbiomas-public/assets/paraguay/collection1/mapbiomas_paraguay_collection1_integration_v1',              #MapBiomas-Paraguay (1985-2022)
                                  'Peru': 'projects/mapbiomas-public/assets/peru/collection2/mapbiomas_peru_collection2_integration_v1',                          #MapBiomas-Peru (1985-2022)
                                  'Uruguay': 'projects/MapBiomas_Pampa/public/collection3/mapbiomas_uruguay_collection1_integration_v1',                          #MapBiomas-Uruguay (1985-2022)
                                  'Venezuela': 'projects/mapbiomas-public/assets/venezuela/collection1/mapbiomas_venezuela_collection1_integration_v1',           #MapBiomas-Venezuela (1985-2022)
                                  'Bolivia': 'projects/mapbiomas-public/assets/bolivia/collection1/mapbiomas_bolivia_collection1_integration_v1',                 #MapBiomas-Bolivia (1985-2021)
                            }                                                                                                                  
                                 
    Soy_dataset_version = 'projects/glad/soy_annual_SA' # Update with latest version Soybean dataset from Song et al. 2020 (https://www.nature.com/articles/s41893-021-00729-z). 
                                                        # Presently contains Soy data till 2022. Data for 2021 is uploaded manually. Sometime the latest year data may have to be inputted manually (Code's inactive part in """   """)

    Cocoa_dataset_version = 'projects/ee-nk-cocoa/assets/cocoa_map_threshold_065'  # Latest dataset from Kalischek et al. 2023 (https://doi.org/10.1038/s43016-023-00751-8)
    Oilpalm_Indonesia_version = 'projects/lu-chandrakant/assets/OilPalm_Indonesia' # Latest dataset from Gaveau et al. 2022 (https://doi.org/10.1371/journal.pone.0266178)
    Oilpalm_Malaysia_version = 'projects/lu-chandrakant/assets/OilPalm_Indonesia_Malaysia'    # Latest dataset from Xu et al. 2020 (https://doi.org/10.5194/essd-12-847-2020)
    Oilpalm_Global_version = "BIOPAMA/GlobalOilPalm/v1"                            # Latest dataset from Descals et al. 2021 (https://doi.org/10.5194/essd-13-1211-2021)
    Coconut_dataset_version = "projects/lu-chandrakant/assets/Coconut"             # Latest dataset from Descals et al. 2023 (https://doi.org/10.5194/essd-15-3991-2023)
    Sugarcane_dataset_version = "projects/lu-chandrakant/assets/Sugarcane"         # Latest dataset from Zheng et al. 2022 (https://doi.org/10.5194/essd-14-2065-2022)
    Rapeseed_dataset_version = "projects/lu-chandrakant/assets/Rapeseed"           # Latest dataset from Han et al. 2021 (https://doi.org/10.5194/essd-13-2857-2021)
    Rice_dataset_version = "projects/lu-chandrakant/assets/Paddy_rice"             # Latest dataset from Han et al. 2021 (https://doi.org/10.5194/essd-13-5969-2021)
    Maize_China_dataset_version = "projects/lu-chandrakant/assets/Maize_China"     # Latest dataset from Peng et al. 2023 (https://doi.org/10.1038/s41597-023-02573-6)
    Rubber_dataset_version = 'users/wangyxtina/MapRubberPaper/rForeRub202122_perc1585DifESAdist5pxPFfinal'     # Latest dataset from Wang et al. 2023 (https://doi.org/10.1038/s41586-023-06642-z)

    Cropland_dataset_version = ['users/potapovpeter/Global_cropland_2003',         # Latest dataset from Potapov et al. 2022 (https://doi.org/10.1038/s43016-021-00429-z)
                                'users/potapovpeter/Global_cropland_2007',         # can be downloaded from https://glad.umd.edu/dataset/croplands
                                'users/potapovpeter/Global_cropland_2011',
                                'users/potapovpeter/Global_cropland_2015',
                                'users/potapovpeter/Global_cropland_2019'
                               ]
        
    Forest_fire_dataset_version = 'users/sashatyu/2001-2022_fire_forest_loss'          # Update Forest fire version https://glad.umd.edu/dataset/Fire_GFL
    Dominant_forest_loss_driver_version = 'projects/lu-chandrakant/assets/Dominant_driver/Dominant_driver_2001_2022'            # Download latest Curtis et al. 2018 (https://www.science.org/doi/10.1126/science.aau3445) dataset from 
                                                                                                                                # https://data.globalforestwatch.org/ and upload to GEE
    Forest_management_dataset_version = "projects/lu-chandrakant/assets/Forest_Management/FML_v3-2_with-colorbar"               # Download latest Lesiv et al. 2022 (https://www.nature.com/articles/s41597-022-01332-3) from https://zenodo.org/record/5848610 and upload to GEE
    
    AGB_dataset_version = ['projects/lu2-chandrakant/assets/Carbon',                   # Download the year 2000 Above ground carbon map from Harris et al. 2021 (https://www.nature.com/articles/s41558-020-00976-6) from 
                           'projects/lu3-chandrakant/assets/Carbon'                    # https://data.globalforestwatch.org/maps/e4bdbe8d6d8d4e32ace7d36a4aec7b93 and upload to GEE. 
                          ]                                                            # Since the AGB dataset is quite large, uploaded using 'geeup' package https://github.com/samapriya/geeup
    
    SOC_stock_dataset_version = "projects/soilgrids-isric/ocs_mean"                    # SoilGrid250m 2.0 (https://soilgrids.org/)
    SOC_content_dataset_version = "projects/soilgrids-isric/soc_mean"                  # First 0-30m Soil Organic Carbon Stock (t/ha)
    Bulk_density_dataset_version = "projects/soilgrids-isric/bdod_mean"                # Latest dataset from Poggio et al. 2021 (https://doi.org/10.5194/soil-7-217-2021) 
    Course_fragment_dataset_version = "projects/soilgrids-isric/cfvo_mean"
    
    Ecoregion_dataset_version = "RESOLVE/ECOREGIONS/2017"                              # To be used for BGB calculation
    Elevation_dataset_version = 'CGIAR/SRTM90_V4'                                      # Check these datasets on Google Earth Engine
    Precipitation_dataset_version = 'UCSB-CHG/CHIRPS/PENTAD'                           # Check these datasets on Google Earth Engine
    Peatland_dataset_version = "projects/lu-chandrakant/assets/Global_Peatlands"       # Peatland dataset can be downloaded from GFW (https://data.globalforestwatch.org/datasets/gfw::global-peatlands/about)
    
    
    ###################
    # 2.1 Check value that are used for commodity classification
    ###################
    
    Sugarcane_MapBiomas_reclass_value = 3221    # Code for reclassified Sugarcane from MapBiomas
    Sugarcane_Brazil_reclass_value = 3222       # Same, but from Zheng et al.
    
    Soy_MapBiomas_reclass_value = 3241          # Code for reclassified Soya beans from MapBiomas
    Soy_SouthAmerica_reclass_value = 3242       # Same, but from Song et al.
    
    Oilpalm_MapBiomas_reclass_value = 6121      # Code for reclassified Oil palm fruit from MapBiomas
    Oilpalm_Plantation_reclass_value = 5121     # Same, but from Du et al.
    Oilpalm_Global_reclass_value = 6122         # Same, but from Descals et al.
    Oilpalm_Indonesia_reclass_value = 6123      # Same, but from Gaveau et al.
    Oilpalm_Malaysia_reclass_value = 6124       # Same, but from Xu et al.
    
    Plantation_base_class_value = 5000          # Reclassified broad plantation class, added with Du et al. dataset
    
    Cocoa_Plantation_reclass_value = 5024       # Code for reclassified Cocoa from Du et al.
    Cocoa_CDGandGhana_reclass_value = 6031      # Same, but from Kalischek et al.
    
    Coconut_Plantation_reclass_value = 5034     # Code for reclassified Coconut from Du et al.
    Coconut_Global_reclass_value = 6041         # Same, but from Descals et al.
    
    Rapeseed_EuropeandCanada_reclass_value = 3301  # Code for reclassified Coconut from Han et al.

    Rice_MapBiomas_reclass_value = 3261         # Code for reclassified Sugarcane from MapBiomas
    Rice_Asia_reclass_value = 3262              # Same, but from Han et al.
    
    Cropland_Global_reclass_value = 3201        # Code for reclassified Cropland from Potopov et al.

    Maize_China_reclass_value = 3321

    Rubber_reclass_values = 6151

    ###################
    # 2.2 Check data for different coutries 
    ###################
    Plantation_dataset_version_country = ['Argentina', 'Australia', 'Brazil', 'Cambodia', 'Cameroon', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Democratic Republic of the Congo', 
                          'Ecuador', 'Gabon', 'Ghana', 'Guatemala', 'Honduras', 'India', 'Indonesia', 'Côte d\'Ivoire', 'Japan', 'Kenya', 'Liberia', 'Malawi', 'Malaysia', 'México', 
                          'Myanmar', 'Nepal', 'New Zealand', 'Nicaragua', 'Nigeria', 'Pakistan', 'Panama', 'Papua New Guinea', 'Peru', 'Philippines', 'Rwanda', 'Solomon Islands', 
                          'South Africa', 'South Korea', 'Sri Lanka', 'Thailand', 'Uruguay', 'United States', 'Venezuela', 'Vietnam',
                          'Åland', 'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Czechia', 'Denmark', 'Estonia', 
                          'Faroe Islands', 'Finland', 'France', 'Germany', 'Greece', 'Guernsey', 'Hungary', 'Iceland', 'Ireland', 'Isle of Man', 'Italy', 'Jersey', 'Kosovo', 
                          'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 
                          'Poland', 'Portugal', 'Romania', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Svalbard and Jan Mayen', 'Sweden', 'Switzerland', 
                          'Ukraine', 'United Kingdom', 'Vatican City']

    MapBiomas_dataset_version_country = ['Argentina','Bolivia','Brazil','Chile','Colombia','Ecuador','French Guiana','Guyana','Paraguay','Peru','Suriname','Uruguay','Venezuela','Indonesia']

    Rapeseed_dataset_version_country = ['Albania','Austria','Bulgaria','Denmark','Belarus','Estonia','Faroe Islands','Finland','France','Germany','Bosnia and Herzegovina',
                          'Greece','Hungary','Croatia','Iceland','Ireland','Italy','Latvia','Lithuania','Malta','Moldova','Netherlands','North Macedonia','Norway',
                          'Czechia','Poland','Portugal','Romania','Slovenia','Slovakia','Spain','Sweden','Switzerland','United Kingdom','Ukraine','Belgium',
                          'Luxembourg','Serbia and Montenegro','Serbia','Montenegro','Åland','Andorra','Guernsey','Isle of Man','Jersey','Kosovo','Liechtenstein',
                          'Monaco','San Marino','Svalbard and Jan Mayen','Vatican City','Turkey','United States','Canada','Chile']

    Rice_dataset_version_country = ['Brunei','Myanmar','Indonesia','Cambodia','Laos','Malaysia','Philippines','Timor-Leste','Singapore','Thailand','Vietnam','China',
                          'Japan','North Korea','South Korea','Taiwan','India']

    Global_oilpalm_and_Coconut_not_in = ['Canada', 'Russia', 'United States', 'Romania', 'Croatia', 'Japan'] 

    
    ####################
    # 3. Load and pre-process datasets
    ####################
    
    geometry = ee.FeatureCollection(GADM_admin_boundary_version) 
    ## Filtering the geometry to a specific country/boundary
    if Admin_boundary in ['United States', 'Russia', 'Canada', 'Brazil']:
        uniqueValues_ADM2CODE = geometry.filter(ee.Filter.eq('COUNTRY', Admin_boundary)).distinct('GID_2').aggregate_array('GID_2');
        uniqueValues_ADM2CODE = uniqueValues_ADM2CODE.sort()
        uniqueValues_ADM2CODE = uniqueValues_ADM2CODE.getInfo()
        # Load the feature collection needed to simulate aggregation
        geometry = geometry.filter(ee.Filter.inList('GID_2', uniqueValues_ADM2CODE)).filter(ee.Filter.eq('COUNTRY', Admin_boundary))
        size = geometry.size().getInfo()
        sub_folder_code = '_'
    else:
        uniqueValues_ADM2CODE = geometry.filter(ee.Filter.eq('COUNTRY', Admin_boundary)).distinct('COUNTRY').aggregate_array('COUNTRY');
        # Load the feature collection needed to simulate aggregation
        geometry = geometry.filter(ee.Filter.inList('COUNTRY', uniqueValues_ADM2CODE)).filter(ee.Filter.eq('COUNTRY', Admin_boundary))
        size = geometry.size().getInfo()
        sub_folder_code = ''
    #$$size = geometry.size().getInfo()
    print('Boundary', Admin_boundary, ':', size)
        
    
    
    ## Load tree cover and tree cover loss dataset (Hansen et al. 2013)
    # Load the latest version of the dataset and select the variables needed to be processed
    Hansen_data = ee.Image(Hansen_tree_cover_dataset_version).select(['treecover2000', 'loss', 'lossyear'])
    Hansen_data = Hansen_data.clip(geometry)


    if Admin_boundary in Plantation_dataset_version_country:
        ## Load global plantation dataset (Du et al. 2022)
        # Define list of image names as uplaoded in GEE
        imageNames = ["pYear_1", "pYear_2", "pYear_3", "pYear_4", "pYear_5", "pYear_6", "pYear_7", "pYear_8", "pYear_9", "pYear_10", "pYear_11", "pYear_12", "pYear_13", "pYear_14"]
        # Map over image names to create a list of images to create an ImageCollection for band 'b1' (i.e, plantyear)
        images = ee.ImageCollection.fromImages(list(map(lambda name: ee.Image(Plantation_dataset_version + name)\
                                                        .select(['b1']).rename(name), imageNames)))
        # Combine all images into one and concatenate all plantation bands into a single band
        image = images.toBands().rename(imageNames)
        concatenatedImage = image.select(imageNames)
        # Use reduce() to apply the reducer function to all bands
        combinedImage = concatenatedImage.reduce(ee.Reducer.max())
        # Mask regions below zero (No-data)
        plantyear = combinedImage.updateMask(combinedImage.gt(0)).select(['max']).rename('plantyear')
    
        # Map over image names to create a list of images to create an ImageCollection for band 'b2' (i.e., startyear) 
        images = ee.ImageCollection.fromImages([ee.Image(Plantation_dataset_version + name)\
                                                .select(['b2']).rename(name) for name in imageNames])
        image = images.toBands().rename(imageNames)
        concatenatedImage = image.select(imageNames)
        combinedImage = concatenatedImage.reduce(ee.Reducer.max())
        startyear = combinedImage.updateMask(combinedImage.gt(0)).select(['max']).rename('startyear')
    
        #Map over image names to create a list of images to create an ImageCollection for band 'b3' (i.e., species)
        images = ee.ImageCollection.fromImages([ee.Image(Plantation_dataset_version + name)\
                                                .select(['b3']).rename(name) for name in imageNames])
        image = images.toBands().rename(imageNames)
        concatenatedImage = image.select(imageNames)
        combinedImage = concatenatedImage.reduce(ee.Reducer.max())
        species = combinedImage.updateMask(combinedImage.gt(0)).select(['max']).rename('species')

    

    if Admin_boundary in MapBiomas_dataset_version_country:
        # Function to retrieve datasets based on the country
        def get_datasets_by_country(Admin_boundary):
            datasets = {
                'Argentina': ['Atlantic forest', 'Argentina', 'Chaco', 'Pampa'],
                'Bolivia': ['Amazon', 'Chaco', 'Bolivia'],
                'Brazil': ['Amazon', 'Atlantic forest', 'Brazil', 'Pampa'],
                'Chile': ['Chile'],
                'Colombia': ['Amazon', 'Colombia'],
                'Ecuador': ['Amazon', 'Ecuador'],
                'French Guiana': ['Amazon'],
                'Guyana': ['Amazon'],
                'Paraguay': ['Atlantic forest', 'Chaco', 'Paraguay'],
                'Peru': ['Amazon', 'Peru'],
                'Suriname': ['Amazon'],
                'Uruguay': ['Uruguay', 'Pampa'],
                'Venezuela': ['Venezuela'],
                'Indonesia': ['Indonesia']
            }
            selected_keys = datasets.get(Admin_boundary, [])
            return [MapBiomas_dataset_version[key] for key in selected_keys if key in MapBiomas_dataset_version]
            
        ## Load MapBiomas dataset
        def combined_img_func(Year):
            # First we reclassify the Mapbiomas defined classes with our classficiation. 'in_class' is Mapbiomas classification and 'reclass' is the our classification (See Supplementary Data)
            in_class = [0,1,2,3,4,5,6,9,10,11,12,13,14,15,18,19,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,57,58,61,62,65,66]
            reclass = [1,1000,1100,1300,2100,2100,2100,5000,2000,2100,2100,2100,3050,4000,3150,3200,3221,3100,2100,2100,600,2100,100,1,2100,700,100,2100,100,100,6121,3800,
                                100,100,3241,3261,3200,2100,2100,2100,2100,6021,6001,3800,2100,2100,3200,3200,2100,3281,3802,2100]
                    
            # We analyse seperately for Indonesia and Latin America due to time-series gap. Latin America datasets are available till 2021 (when simulating this code), 
            # (contd.) while Indonesia is avaible only till 2019.   
            imageNames = get_datasets_by_country(Admin_boundary)
            if Admin_boundary == 'Bolivia':
                # Map over image names to create a list of images
                def func_pxw(imageNames):
                    # We gapfill for year 2020 and 2021 since the Indonesia data in unavaiable then by extending 2019 dataset to end year
                    # Change '2020' if the latest dataset becomes avaible at a later date. 
                    if (Year >= 2022) & (Year <= end_year):
                        image = ee.Image(imageNames).select(['classification_2021'])
                    else:
                        image = ee.Image(imageNames).select(['classification_' + str(Year)])
                    return image
                images = ee.ImageCollection.fromImages([func_pxw(imageNames).rename('Class').remap(in_class, reclass, 1) for name in imageNames])
            else:
                # Map over image names to create a list of images
                images = ee.ImageCollection.fromImages([ee.Image(name).select(['classification_' + str(Year)]).rename('Class').remap(in_class, reclass, 1) for name in imageNames])
                        
                        
            # Combine all images into one and concatenate all plantation bands into a single band
            image = images.toBands().rename(imageNames)
            concatenatedImage = image.select(imageNames)
            # Use reduce() to apply the reducer function to all bands
            combinedImage = concatenatedImage.reduce(ee.Reducer.max());
            #combinedImage = combinedImage.clip(geometry)#$$
            # We take maximum so that if a pixel has multiple values
            MapBiomas = combinedImage.select(['max']).rename('classification');
            return MapBiomas
        
        
        ## Load Soybean dataset
        # Load the existing ImageCollection
        soy_collection = ee.ImageCollection(Soy_dataset_version)
    
        def Soy_combined_img_func(Year):
            year = Year
            year_string = ee.String(str(year)) # convert year to string
            # filter the soy_collection by the specified year
            filtered_collection = soy_collection.filterDate(
                year_string.cat('-01-01'), # start of year
                year_string.cat('-12-31')  # end of year
            )
            # Use reduce() to apply the reducer function to all bands
            combinedImage = filtered_collection.reduce(ee.Reducer.max())
            #combinedImage = combinedImage.clip(geometry)#$$
            combinedImage = combinedImage.updateMask(combinedImage.eq(1))
            # Mask regions below zero (No-data)
            soy = combinedImage.select(['b1_max']).rename('classification').multiply(Soy_SouthAmerica_reclass_value)
            return soy


    if Admin_boundary == 'China':
        ## Load Maize-China (Peng et al. 2023)
        # Specify the asset folder path where Maize features are stored
        maize_china_list = ee.data.getList({'id': Maize_China_dataset_version})
        maize_china_list = list(map(lambda image: image['id'].split("/")[4], maize_china_list))
        
        # Map over image names to create an image collection for lu2-chandrakant
        maize_china = ee.ImageCollection(list(map(lambda name: ee.Image(Maize_China_dataset_version+"/" + name).select(['b1']).rename('Class'), maize_china_list)))
        
        def maize_china_func(Year):
            # Print the updated ImageCollection
            if Year > 2020:
                year = 2020
            else:
                year = Year
            year_string = ee.String(str(year))  # Convert year to string
        
            # Filter the maize_china collection by the specified year
            filtered_collection = maize_china.filterDate(
                year_string.cat('-01-01T01:01:01'),  # Start of year
                year_string.cat('-12-31T12:01:01')  # End of year
            )
            # Use reduce() to apply the reducer function to all bands
            combined_image = filtered_collection.reduce(ee.Reducer.max())
            #combined_image = combined_image.clip(geometry)#$$
            combined_image = combined_image.updateMask(combined_image.eq(1))
            # Mask regions below zero (No-data)
            maize = combined_image.select(['Class_max']).rename('classification')
            return maize.multiply(Maize_China_reclass_value)


    if Admin_boundary in ['Côte d\'Ivoire', 'Ghana']: 
        ## Load Cocoa dataset 
        Cocoa_map_CDG = ee.Image(Cocoa_dataset_version)
        #Cocoa_map_CDG = Cocoa_map_CDG.clip(geometry)#$$
        Cocoa_map_CDG_projection =  ee.Image(Cocoa_map_CDG).projection()


    if Admin_boundary == 'Indonesia':
        ## Load Oilpalm Indonesia dataset
        assetList = ee.data.getList({'id': Oilpalm_Indonesia_version})
        assetIds = [asset['id'] for asset in assetList]
        
        OilPalm_Indonesia_org = ee.FeatureCollection([])
        for assetId in assetIds:
            assetCollection = ee.FeatureCollection(assetId)
            OilPalm_Indonesia_org = OilPalm_Indonesia_org.merge(assetCollection)



    if Admin_boundary == 'Malaysia':
        ## Load (Malaysia) Oilpalm dataset
        # Specify the asset folder path where Oil palm features are stored
        OilPalm_Malaysia_list = ee.data.getList({'id': Oilpalm_Malaysia_version})
        OilPalm_Malaysia_list = [image['id'].split('/')[4] for image in OilPalm_Malaysia_list]
        
        # Map over image names to create an image collection for lu2-chandrakant
        OilPalm_Malaysia = ee.ImageCollection([ee.Image(Oilpalm_Malaysia_version+ "/" + name).select(['b1']).rename('Class') for name in OilPalm_Malaysia_list])
        
        
        def OilPalm_Malaysia_func(Year):
            # Print the updated ImageCollection
            if Year > 2018:
                year = 2018
            else:
                year = Year
        
            yearString = ee.String(str(year))  # convert year to string
            # filter the OilPalm_Malaysia collection by the specified year
            filteredCollection = OilPalm_Malaysia.filterDate(
                yearString.cat('-01-01'),  # start of year
                yearString.cat('-12-31')  # end of year
            )
        
            # Use reduce() to apply the reducer function to all bands
            combinedImage = filteredCollection.reduce(ee.Reducer.max())
            #combinedImage = combinedImage.clip(geometry)#$$
            combinedImage = combinedImage.updateMask(combinedImage.eq(1))
        
            # Mask regions below zero (No-data)
            OilPalm = combinedImage.select(['Class_max']).rename('classification')
            return OilPalm.multiply(Oilpalm_Malaysia_reclass_value)


    if Admin_boundary not in Global_oilpalm_and_Coconut_not_in:
        ## Load (Global) Oilpalm dataset
        OilPalm_Global = ee.ImageCollection(Oilpalm_Global_version)
        OilPalm_Global_projection =  ee.Image(OilPalm_Global.first()).projection()      # Will be needed later for ee.reduceResolution
        OilPalm_Global = OilPalm_Global.reduce(ee.Reducer.min())
        #OilPalm_Global = OilPalm_Global.clip(geometry)#$$
        OilPalm_Global = OilPalm_Global.updateMask(OilPalm_Global.neq(3))
        OilPalm_Global = OilPalm_Global.select(['classification_min']).rename(['Class'])
    
    
    
        ## Load Coconut dataset
        Coconut_list = ee.data.getList({'id': Coconut_dataset_version})
        Coconut_list = [((image['id']).split("/")[4]) for image in Coconut_list]
        Coconut = ee.ImageCollection.fromImages([ee.Image(Coconut_dataset_version+"/" + name).select(['b1']).rename('Class') for name in Coconut_list])
        Coconut_projection =  ee.Image(Coconut.first()).projection()
        Coconut = Coconut.reduce(ee.Reducer.min()).rename('Class')
        #Coconut = Coconut.clip(geometry)#$$
        Coconut = Coconut.updateMask(Coconut.eq(1))
    

    """
    ## Load Rubber dataset (Wang et al. 2023)
    Rubber = ee.Image(Rubber_dataset_version)
    #Rubber = Rubber.clip(geometry)#$$
    Rubber_projection =  ee.Image(Rubber).projection()
    Rubber = Rubber.updateMask(Rubber.eq(2)) # 1-Forest and 2-Rubber
    Rubber = Rubber.where(Rubber, 1)
    """
    
    
    ## Load Cropland datasets
    Cropland_2000_03 = ee.ImageCollection(Cropland_dataset_version[0]).reduce(ee.Reducer.max()).rename('Class')    
    Cropland_2004_07 = ee.ImageCollection(Cropland_dataset_version[1]).reduce(ee.Reducer.max()).rename('Class')    
    Cropland_2008_11 = ee.ImageCollection(Cropland_dataset_version[2]).reduce(ee.Reducer.max()).rename('Class')    
    Cropland_2012_15 = ee.ImageCollection(Cropland_dataset_version[3]).reduce(ee.Reducer.max()).rename('Class')    
    Cropland_2016_19 = ee.ImageCollection(Cropland_dataset_version[4]).reduce(ee.Reducer.max()).rename('Class')


    if Admin_boundary == 'Brazil':
        ## Load Sugarcane Brazil dataset
        Sugarcane_list = ee.data.getList({'id': Sugarcane_dataset_version})
        Sugarcane_list = [((image['id']).split("/")[4]) for image in Sugarcane_list]
        Sugarcane_Brazil = ee.ImageCollection.fromImages([ee.Image(Sugarcane_dataset_version+"/" + name).select(['b1']).rename('Class') for name in Sugarcane_list])
        Sugarcane_Brazil = Sugarcane_Brazil.reduce(ee.Reducer.max()).rename('Class')
        #Sugarcane_Brazil = Sugarcane_Brazil.clip(geometry)#$$
        Sugarcane_Brazil = Sugarcane_Brazil.updateMask(Sugarcane_Brazil.eq(1))

    

    if Admin_boundary in Rapeseed_dataset_version_country:
        ## Load Rapeseed dataset
        Rapeseed_list = ee.data.getList({'id': Rapeseed_dataset_version})
        Rapeseed_list = [((image['id']).split("/")[4]) for image in Rapeseed_list]
        Rapeseed = ee.ImageCollection.fromImages([ee.Image(Rapeseed_dataset_version+"/" + name).select(['b1']).rename('Class') for name in Rapeseed_list])
        Rapeseed_projection =  ee.Image(Rapeseed.first()).projection()
        Rapeseed = Rapeseed.reduce(ee.Reducer.max()).rename('Class')
        #Rapeseed = Rapeseed.clip(geometry)#$$
        Rapeseed = Rapeseed.updateMask(Rapeseed.eq(1))


    if Admin_boundary in Rice_dataset_version_country:
        ## Load Paddy rice dataset
        Paddy_rice_list = ee.data.getList({'id': Rice_dataset_version})
        Paddy_rice_list = [((image['id']).split("/")[4]) for image in Paddy_rice_list]
        Paddy_rice = ee.ImageCollection.fromImages([ee.Image(Rice_dataset_version+"/" + name).select(['b1']).rename('Class') for name in Paddy_rice_list])
        Paddy_rice_projection =  ee.Image(Paddy_rice.first()).projection()
        Paddy_rice = Paddy_rice.reduce(ee.Reducer.max()).rename('Class')
        #Paddy_rice = Paddy_rice.clip(geometry)#$$
        Paddy_rice = Paddy_rice.updateMask(Paddy_rice.eq(1))



    ## Load Forest management data
    ForestManagement = ee.Image(Forest_management_dataset_version);
    #ForestManagement = ForestManagement.clip(geometry)#$$
    
    
    ## Load forest loss due to fire
    Forest_fire = ee.ImageCollection(Forest_fire_dataset_version)
    # 1-No fire, 2- Low certainity, 3-Medium certainity, 4-High certainity and 5-'all forest loss due to fire in Africa'
    in_class = [1, 2, 3, 4, 5]
    reclass = [1, 1, 250, 250, 250]
    Forest_fire = Forest_fire.reduce(ee.Reducer.min()).select(['b1_min']).rename('classification')
    #Forest_fire = Forest_fire.clip(geometry)#$$
    Forest_fire = Forest_fire.remap(in_class, reclass, 1)

    
    
    ## Load Dominant tree cover loss driver
    dominant_driver = ee.Image(Dominant_forest_loss_driver_version)
    #dominant_driver = dominant_driver.clip(geometry)#$$
    in_class = [1, 2, 3, 4, 5] 
    # These classes mean 1-Commodity-driven deforestation, 2-Shifting agriculture, 3-Forestry, 4-Wildfire, and 5-Urbanization
    # Reclassify them to our classification (See Supplementary Data)
    reclass = [3000, 3000, 500, 200, 600] 
    dominant_driver = dominant_driver.remap(in_class, reclass, 1)

    
    
    ## Load Above ground biomass dataset (Harris et al. 2018)
    if simulation in ['All', 'AGB', 'BGB']:
        # Get the list of images in the lu2-chandrakant asset (i.e., uploaded to GEE)
        lu2_asset_list = ee.data.getList({'id': AGB_dataset_version[0]})
        lu2_image_names = [image['id'].split('/')[-1] for image in lu2_asset_list]
        # Map over image names to create an image collection for lu2-chandrakant
        lu2_images = ee.ImageCollection.fromImages([ee.Image(AGB_dataset_version[0]+f'/{name}').select(['b1']).rename(name) for name in lu2_image_names])
        # Get the list of images in the lu3-chandrakant asset
        lu3_asset_list = ee.data.getList({'id': AGB_dataset_version[1]})
        lu3_image_names = [image['id'].split('/')[-1] for image in lu3_asset_list]
        # Map over image names to create an image collection for lu3-chandrakant
        lu3_images = ee.ImageCollection.fromImages([ee.Image(AGB_dataset_version[1]+f'/{name}').select(['b1']).rename(name) for name in lu3_image_names])
    
        # Merge the two image collections into one
        all_images = lu2_images.merge(lu3_images)
        # Combine all images into one and concatenate all plantation bands into a single band
        image = all_images.toBands().rename(lu2_image_names + lu3_image_names)
        concatenated_image = image.select(lu2_image_names + lu3_image_names)
        # Use reduce() to apply the reducer function to all bands
        combined_image = concatenated_image.reduce(ee.Reducer.max())
        #combined_image = combined_image.clip(geometry)#$$
        # Mask regions below zero (No-data)
        AGB = combined_image.updateMask(combined_image.gte(0)).select(['max']).rename('MG_px-1')


        ## Load elevation data and apply mask
        elevation = ee.Image(Elevation_dataset_version).select('elevation')
        #elevation = elevation.clip(geometry)#$$
        ## Load and filter precipitation data
        precipitation = ee.ImageCollection(Precipitation_dataset_version) \
            .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
            .select('precipitation') \
            .reduce(ee.Reducer.sum())
        #precipitation = precipitation.clip(geometry)#$$
    

    
    ## Load SOC database
    if simulation in ['All', 'SOC 0-30', 'SOC 30-100']:
        # Function to calculate SOC for a given depth range
        # Combine SOC layers for different depth ranges
        SOC_0_30 = ee.Image(SOC_stock_dataset_version).select("ocs_0-30cm_mean");    # SOC (t/ha) is already derived from SoilGrid
        #SOC_0_30 = SOC_0_30.clip(geometry)#$$
        def calculate_SOC(depthRange1, depthRange2):
            soc = ee.Image(SOC_content_dataset_version).select("soc_" + str(depthRange1) + '-' + str(depthRange2) + "cm_mean")
            bd = ee.Image(Bulk_density_dataset_version).select("bdod_" + str(depthRange1) + '-' + str(depthRange2) + "cm_mean")
            cf = ee.Image(Course_fragment_dataset_version).select("cfvo_" + str(depthRange1) + '-' + str(depthRange2) + "cm_mean")
            cf = (cf.multiply(1e-3).subtract(1)).multiply(-1)
            soc_t_per_ha = (soc.multiply(bd).multiply(1e-5).multiply(depthRange2-depthRange1)).multiply(cf).rename('SOC t ha-1');
            #soc_t_per_ha = soc_t_per_ha.clip(geometry)#$$
            return soc_t_per_ha





    ## Load Global Peatland dataset
    if simulation in ['All', 'PEATLAND', 'SOC 0-30', 'SOC 30-100']:
        # Get a list of images in the asset collection
        Peatland_list = ee.data.getList({'id': Peatland_dataset_version})
        # Extract image names from the list
        Peatland_list = [image['id'].split('/')[-1] for image in Peatland_list]
        # Map over image names to create an image collection
        Peatland = ee.ImageCollection([ee.Image(f"{Peatland_dataset_version}/{name}").select(['b1']).rename('Class') for name in Peatland_list])
        # Reduce the image collection by taking the maximum pixel value
        Peatland = Peatland.reduce(ee.Reducer.max()).rename('Class')
        #Peatland = Peatland.clip(geometry)#$$
        # Select pixels with a value of 1 (representing Peatland)
        Peatland = Peatland.updateMask(Peatland.eq(1))



    
    
    ####################
    # 4. Analysing dataset
    ####################
    
    ## Analysing forest loss dataset
    # List sequence required for processing. Here '1' refers to 2001.
    #years = ee.List.sequence(1, end_year-2000)
    # Extract the scale of the Hansen variable. This will be needed for resampling other datasets.
    Hansen_scale = Hansen_data.projection().nominalScale()
    # Calculate the tree cover area
    Hansen_treeCover = Hansen_data.select(['treecover2000'])
    Hansen_loss = Hansen_data.select(['loss'])
    Hansen_lossyear = Hansen_data.select(['lossyear'])

    # Hansen projection
    Hansen_projection = ee.Image(Hansen_loss).projection()
    
    # Restricting forest loss to Forest_threshold
    Hansen_treeCover = Hansen_treeCover.updateMask(Hansen_treeCover.gte(Forest_threshold))
    TC_mask = Hansen_loss.gt(0).And(Hansen_treeCover.gt(0))
    #Updating mask
    Hansen_data = Hansen_data.updateMask(TC_mask)
    Hansen_loss = Hansen_loss.updateMask(TC_mask)

    # Hansen's data has the treecover2000 layer ranging from 0-100.
    Hansen_treeCover = Hansen_treeCover.divide(100)
    # It needs to be multiplied by 10^(-4) to convert the areas from 'm2' to 'ha'.
    #area_HansenLoss = Hansen_loss.gt(0).multiply(ee.Image.pixelArea()).divide(1e4).select([0], ["arealoss"])

    
    if Admin_boundary in MapBiomas_dataset_version_country:
        ## Analysing MapBiomas dataset
        # Define the years list
        years = [year for year in range(2001, end_year+1)]
        # Define a function to get MapBiomas data for a given year and apply a mask (i.e., mask removes all regions where forest loss isn't happening)
        def getMapBiomas(year):
            # Load the MapBiomas data for the given year
            MapBiomas = combined_img_func(year).updateMask(TC_mask)
            # Return the masked MapBiomas image with the year as a property
            return MapBiomas.set('year', year)
    
        # Map the getMapBiomas function over the years array to create an ImageCollection
        MapBiomas_collection = ee.ImageCollection.fromImages(
            list(map(getMapBiomas, years))
        )
        
        MapBiomasyear = ee.List.sequence(2001, end_year, 1)
        # Sequencing over only one year (i.e., forest loss year) and extracting values to MapBiomas pixel
        """
        # Map over image names to create a list of images
        images = ee.ImageCollection.fromImages(
            MapBiomasyear.map(
                lambda name: ee.Image(
                    MapBiomas_collection.filterMetadata('year', 'equals', ee.Number(name)).first())
                .select('classification')
                .updateMask(Hansen_lossyear.eq(ee.Number(name).subtract(2000)))
                .rename('classification'))
        )
        # Use reduce() to apply the reducer function to all bands
        MapBiomas = images.reduce(ee.Reducer.max()).select(['classification_max']).rename('classification')
        """
        
        # Sequencing over only four year (i.e., forest loss year + three proceeding years) and extracting maximum pixel value (i.e., commodity-driven)
        # This way, plantation will be preferred over perrinial crops, and perrinial crops preferred over temperory crops
        # Map over image names to create a list of images
        images = ee.ImageCollection.fromImages(MapBiomasyear.map(lambda name: 
            ee.ImageCollection(MapBiomas_collection).filter(ee.Filter.inList('year', ee.List.sequence(ee.Number(name), ee.Number(name).add(3), 1)))\
                .reduce(ee.Reducer.max()).select(['classification_max'])\
                .rename('classification')\
                .updateMask(Hansen_lossyear.eq(ee.Number(name).subtract(2000)))\
                .rename('classification')
        ))
        # Use reduce() to apply the reducer function to all bands
        MapBiomas = images.reduce(ee.Reducer.max()).select(['classification_max']).rename('classification')
        # Matching our pixels with those observed for year 2000
        MapBiomas_2000 = combined_img_func(2000).updateMask(TC_mask)
        # If MapBiomas 2000 is equal to MapBiomas(year), the pixel value is multiplied by -1 to suggest pre-2000's disturbance
        MapBiomas = MapBiomas.where(MapBiomas.eq(MapBiomas_2000), MapBiomas.multiply(-1))
        # Adding this line to remove negative 1 values (since 1 is classified as unknown)
        MapBiomas = MapBiomas.where(MapBiomas.eq(-1), 1)
    
        
        
        ## Analysing Soybean dataset    
        years = [year for year in range(2001, end_year+1)]
        # Create an empty ImageCollection to store Soya beans annual dataset
        Soy_collection = ee.ImageCollection([])
        # Define a function to get Soya beans data for a given year and apply a mask
        def getSoy(year):
            # Load the Soya beans data for the given year
            Soy = ee.Image(Soy_combined_img_func(year)).updateMask(TC_mask)
            # Return the masked Soya beans image with the year as a property
            return Soy.set('year', year)
        # Map the getSoy function over the years array to create an ImageCollection
        Soy_collection = ee.ImageCollection.fromImages(
            list(map(getSoy, years))
        )
        
        Soyyear = ee.List.sequence(2001, end_year, 1)
        # The code below is sequncing over only one year (i.e., forest loss year)
        """
        # The year of forest loss was same as Hansen forest loss
        def get_masked_Soy(name):
            year = ee.Number(name) # Convert name to a number
            image = Soy_collection.filterMetadata('year', 'equals', year).first()
            maskedImage = ee.Image(image).select('classification').updateMask(Hansen_lossyear.eq(year.subtract(2000)))
            return maskedImage.rename('classification')
    
        # Map the get_masked_Soy function over the years array to create an ImageCollection
        images = ee.ImageCollection.fromImages(Soyyear.map(get_masked_Soy))
        # Use reduce() to apply the reducer function to all bands
        Soy = images.reduce(ee.Reducer.max()).select(['classification_max']).rename('classification')
        """
        
        # The code below is sequncing over four year (i.e., forest loss year + three proceeding years)
        # Map the Soy_collection function over the years array to create an ImageCollection
        def get_four_years_Soy(name):
            year = ee.Number(name) # Convert name to a number
            years = ee.List.sequence(year, year.add(3), 1)
            filter = ee.Filter.inList('year', years)
            image = Soy_collection.filter(filter)
            image = image.reduce(ee.Reducer.max()).select(['classification_max']).rename('classification')
            maskedImage = ee.Image(image).select('classification').updateMask(Hansen_lossyear.eq(year.subtract(2000)))
            return maskedImage.rename('classification')
        images = ee.ImageCollection.fromImages(Soyyear.map(get_four_years_Soy))
        # Use reduce() to apply the reducer function to all bands
        Soy_four = images.reduce(ee.Reducer.max()).select(['classification_max']).rename('classification')

    

    if Admin_boundary == 'China':
        # Analyse Maize-China dataset
        # Define a list of years
        years = [year for year in range(2001, end_year+1)]
        
        # Define a function to get Maize-China data for a given year and apply a mask
        def get_maize(year):
            # Load the Maize-China data for the given year
            maize = maize_china_func(year).updateMask(TC_mask)
            # Return the masked Maize-China image with the year as a property
            return maize.set('year', year)
        
        # Map the get_maize function over the years list to create an ImageCollection
        maize_china_collection = ee.ImageCollection(list(map(get_maize, years)))
        
        # Define a list of years for processing
        maize_years = ee.List.sequence(2001, end_year, 1)
    
        # Map the get_maize function over the maize_years list to create an ImageCollection
        def get_four_years_Maize(name):
            year = ee.Number(name) # Convert name to a number
            years = ee.List.sequence(year, year.add(3), 1)
            filter = ee.Filter.inList('year', years)
            image = maize_china_collection.filter(filter)
            image = image.reduce(ee.Reducer.max()).select(['classification_max']).rename('classification')
            maskedImage = ee.Image(image).select('classification').updateMask(Hansen_lossyear.eq(year.subtract(2000)))
            return maskedImage.rename('classification')
        images = ee.ImageCollection.fromImages(maize_years.map(get_four_years_Maize))
        # Use reduce() to apply the reducer function to all bands
        Maize_China_four = images.reduce(ee.Reducer.max()).select(['classification_max']).rename('classification')

        

    if Admin_boundary == 'Indonesia':
        ## Analyse Oilpalm Indonesia dataset    
        # Filter out non-forest and smallholder plantations in year 2000
        OilPalm_Indonesia = OilPalm_Indonesia_org.filter(
          ee.Filter.And(
            ee.Filter.neq('F2000', 'IOPP'),
            ee.Filter.neq('F2000', 'Smallholder')
          ))
        OilPalm_Indonesia_pre2000 = OilPalm_Indonesia_org.filter(
          ee.Filter.Or(
            ee.Filter.eq('F2000', 'IOPP'),
            ee.Filter.eq('F2000', 'Smallholder')
          ))
        
        # Add the new property to each feature in the feature collection
        def set_reclass_code(feature):
            return feature.set('Reclass Code', Oilpalm_Indonesia_reclass_value)
        
        OilPalm_Indonesia = OilPalm_Indonesia.map(set_reclass_code)
        OilPalm_Indonesia_pre2000 = OilPalm_Indonesia_pre2000.map(set_reclass_code)
        OilPalm_Indonesia_Image = OilPalm_Indonesia.reduceToImage(
          properties=['Reclass Code'],        # replace 'property_name' with the name of the property in your shapefile that contains the value you want to use
          reducer=ee.Reducer.sum()
        )
        OilPalm_Indonesia_pre2000_Image = OilPalm_Indonesia_pre2000.reduceToImage(
          properties=['Reclass Code'],
          reducer=ee.Reducer.sum()
        )
        # Clip and update masks
        #OilPalm_Indonesia_Image = OilPalm_Indonesia_Image.clip(geometry)#$$
        #OilPalm_Indonesia_pre2000_Image = OilPalm_Indonesia_pre2000_Image.clip(geometry)#$$
    
        OilPalm_Indonesia_Image = OilPalm_Indonesia_Image.updateMask(TC_mask)
        OilPalm_Indonesia_pre2000_Image = OilPalm_Indonesia_pre2000_Image.updateMask(TC_mask)


    
    ## Analyze Cropland dataset
    # Selecting the dataset within the appropriate year range
    Cropland_2000_03 = Cropland_2000_03.updateMask(Cropland_2000_03.eq(1).And(Hansen_lossyear.gte(1)).And(Hansen_lossyear.lte(3)))
    Cropland_2004_07 = Cropland_2004_07.updateMask(Cropland_2004_07.eq(1).And(Hansen_lossyear.gte(1)).And(Hansen_lossyear.lte(7)))
    Cropland_2008_11 = Cropland_2008_11.updateMask(Cropland_2008_11.eq(1).And(Hansen_lossyear.gte(5)).And(Hansen_lossyear.lte(11)))
    Cropland_2012_15 = Cropland_2012_15.updateMask(Cropland_2012_15.eq(1).And(Hansen_lossyear.gte(8)).And(Hansen_lossyear.lte(15)))
    Cropland_2016_19 = Cropland_2016_19.updateMask(Cropland_2016_19.eq(1).And(Hansen_lossyear.gte(12)).And(Hansen_lossyear.lte(19)))
    
    # Create an ImageCollection with the cropland expansion datasets
    combinedCropland = ee.ImageCollection([
      Cropland_2000_03.updateMask(TC_mask),
      Cropland_2004_07.updateMask(TC_mask),
      Cropland_2008_11.updateMask(TC_mask),
      Cropland_2012_15.updateMask(TC_mask),
      Cropland_2016_19.updateMask(TC_mask)
    ])
    # Mosaic the ImageCollection to create a single image
    combinedCropland = combinedCropland.mosaic()
    # Reduce and rename the combined image
    combinedCropland = combinedCropland.reduce(ee.Reducer.max()).rename('Class')
    #combinedCropland = combinedCropland.clip(geometry)#$$
    combinedCropland = combinedCropland.multiply(Cropland_Global_reclass_value)
    


    if Admin_boundary == 'Brazil':
        ## Analyze Sugarcane Brazil dataset
        Sugarcane_Brazil = Sugarcane_Brazil.updateMask(TC_mask)
        Sugarcane_Brazil = Sugarcane_Brazil.where(Sugarcane_Brazil.gt(0), Sugarcane_Brazil_reclass_value)

    

    if Admin_boundary in Rapeseed_dataset_version_country:
        ## Analyze Rapeseed dataset
        Rapeseed = Rapeseed.updateMask(TC_mask)
        # Set a default projection
        Rapeseed = Rapeseed.setDefaultProjection(Rapeseed_projection)
        Rapeseed = Rapeseed.reduceResolution(reducer = ee.Reducer.count(), bestEffort = True, maxPixels = 3*3).reproject(Hansen_projection)
        Rapeseed = Rapeseed.updateMask(Rapeseed.gt(4))       # Reduce resolution based on majority interpolation 
        Rapeseed = Rapeseed.where(Rapeseed.gt(0), Rapeseed_EuropeandCanada_reclass_value)



    if Admin_boundary in Rice_dataset_version_country:
        ## Analyze Paddy rice dataset
        Paddy_rice = Paddy_rice.updateMask(TC_mask)
        Paddy_rice = Paddy_rice.setDefaultProjection(Paddy_rice_projection)
        Paddy_rice = Paddy_rice.reduceResolution(reducer = ee.Reducer.count(), bestEffort = True, maxPixels = 3*3).reproject(Hansen_projection)
        Paddy_rice = Paddy_rice.updateMask(Paddy_rice.gt(4))       # Reduce resolution based on majority interpolation 
        Paddy_rice = Paddy_rice.where(Paddy_rice.gt(0), Rice_Asia_reclass_value)
    
    

    if Admin_boundary in Plantation_dataset_version_country:
        ## Analysing Plantation dataset
        # A value of '1981' means that the planting year was before 1982
        plantyear = plantyear.updateMask(TC_mask)
        startyear = startyear.updateMask(TC_mask)
        startyear = startyear.updateMask(plantyear.gte(1))
        startyear = startyear.updateMask(plantyear.gt(startyear)) # Plant year shouldn't be less than start year
        species = species.updateMask(TC_mask).updateMask(startyear)
        
        # Seperating pre- and post-2000's deforestation using startyear
        FL_att_to_new_plantations = startyear.updateMask(startyear.gt(Plantation_threshold_year))
        FL_att_to_old_plantations = startyear.updateMask(startyear.lte(Plantation_threshold_year))
        FL_att_to_new_plantations_species = species.updateMask(FL_att_to_new_plantations.gt(0))
        FL_att_to_old_plantations_species = species.updateMask(FL_att_to_old_plantations.gt(0))
    
        # Maching with the classification of MapBiomas-reclass
        FL_att_to_old_plantations_species = FL_att_to_old_plantations_species.add(Plantation_base_class_value)
        FL_att_to_new_plantations_species = FL_att_to_new_plantations_species.add(Plantation_base_class_value)

    
    
    
    ## Complimenting Curtis et al with Forest management dataset to screen pre- and post-2000 deforestion byu removing already managed forests
    ForestManagement = ForestManagement.updateMask(TC_mask)
    ManagedForests_org = ForestManagement.updateMask(ForestManagement.eq(20)                          # 20: Naturally regenerating forest with signs of management, e.g., logging, clear cuts etc;
                                                 .Or(ForestManagement.eq(31))                         # 31: Planted forests;
                                                 .Or(ForestManagement.eq(32))                         # 32: Plantation forests (rotation time up to 15 years);
                                                 .Or(ForestManagement.eq(40))                         # 40: Oil palm plantations;
                                                 .Or(ForestManagement.eq(53)))                        # 53: Agroforestry;

    # Check if Admin_boundary is in the country list (i.e., Du et al. Forest plantatation data is preferred)
    if Admin_boundary in Plantation_dataset_version_country:
        ManagedForests = FL_att_to_old_plantations_species
    else: #Otherwise Managed Forest dataset is preferred 
        ManagedForests = ManagedForests_org



    if Admin_boundary in ['Côte d\'Ivoire', 'Ghana']: 
        ## Analyze Cocoa dataset
        Cocoa_map_CDG = Cocoa_map_CDG.updateMask(TC_mask)
        Cocoa_map_CDG = Cocoa_map_CDG.setDefaultProjection(Cocoa_map_CDG_projection)
        Cocoa_map_CDG = Cocoa_map_CDG.reduceResolution(reducer = ee.Reducer.count(), bestEffort = True, maxPixels = 3*3).reproject(Hansen_projection)
        Cocoa_map_CDG = Cocoa_map_CDG.updateMask(Cocoa_map_CDG.gt(4))       # Reduce resolution based on majority interpolation 
        Cocoa_map_CDG = Cocoa_map_CDG.where(Cocoa_map_CDG.gt(0), Cocoa_CDGandGhana_reclass_value)
        Cocoa_map_CDG = Cocoa_map_CDG.where(ManagedForests.gt(0).And(Cocoa_map_CDG.gt(1)), Cocoa_map_CDG.multiply(-1))
    
    


    if Admin_boundary == 'Malaysia': 
        ## Analyze (Malaysia) Oilpalm dataset
        years = [year for year in range(2001, end_year+1)]
        # Define a function to get OilPalm data for a given year and apply a mask
        def getOilPalm(year):
            # Load the OilPalm data for the given year
            OilPalm = OilPalm_Malaysia_func(year).updateMask(TC_mask)
            # Return the masked OilPalm image with the year as a property
            return OilPalm.set('year', year)
        
        # Map the getOilPalm function over the years list to create an ImageCollection
        OilPalm_Malaysia_collection = ee.ImageCollection.fromImages(
            list(map(getOilPalm, years))
        )
        
        # Create a list of years for processing
        OilPalmyear = ee.List.sequence(2001, end_year, 1)
        
        # Map the getOilPalm function over the years list to create an ImageCollection
        def get_four_years_OilPalm(name):
            year = ee.Number(name) # Convert name to a number
            years = ee.List.sequence(year, year.add(3), 1)
            filter = ee.Filter.inList('year', years)
            image = OilPalm_Malaysia_collection.filter(filter)
            image = image.reduce(ee.Reducer.max()).select(['classification_max']).rename('classification')
            maskedImage = ee.Image(image).select('classification').updateMask(Hansen_lossyear.eq(year.subtract(2000)))
            return maskedImage.rename('classification')
        images = ee.ImageCollection.fromImages(OilPalmyear.map(get_four_years_OilPalm))
    
        # Use reduce() to apply the reducer function to all bands
        OilPalm_Malaysia_four = images.reduce(ee.Reducer.max()).select(['classification_max']).rename('classification')
        OilPalm_Malaysia_four = OilPalm_Malaysia_four.where(ManagedForests.gt(0).And(OilPalm_Malaysia_four.gt(0)), OilPalm_Malaysia_four.multiply(-1))

    
    if Admin_boundary not in Global_oilpalm_and_Coconut_not_in:
        ## Analyze (Global) Oilpalm dataset
        OilPalm_Global = OilPalm_Global.updateMask(TC_mask)
        OilPalm_Global = OilPalm_Global.setDefaultProjection(OilPalm_Global_projection)
        OilPalm_Global = OilPalm_Global.reduceResolution(reducer = ee.Reducer.count(), bestEffort = True, maxPixels = 3*3).reproject(Hansen_projection)
        OilPalm_Global = OilPalm_Global.updateMask(OilPalm_Global.gt(4))       # Reduce resolution based on majority interpolation 
        OilPalm_Global = OilPalm_Global.where(OilPalm_Global.gt(0), Oilpalm_Global_reclass_value)
        OilPalm_Global = OilPalm_Global.where(ManagedForests.gt(0).And(OilPalm_Global.gt(0)), OilPalm_Global.multiply(-1))
        
        
        ## Analyze Coconut dataset
        Coconut = Coconut.updateMask(TC_mask)
        Coconut = Coconut.setDefaultProjection(Coconut_projection)
        Coconut = Coconut.reduceResolution(reducer = ee.Reducer.count(), bestEffort = True, maxPixels = 3).reproject(Hansen_projection)
        Coconut = Coconut.updateMask(Coconut.gte(1))       # Reduce resolution based on majority interpolation (45% majority for 20 m resolution)
        Coconut = Coconut.where(Coconut.gt(0), Coconut_Global_reclass_value)
        Coconut = Coconut.where(ManagedForests.gt(0).And(Coconut.gt(0)), Coconut.multiply(-1))
    

    """
    # Load Rubber dataset
    Rubber = Rubber.updateMask(TC_mask)
    Rubber = Rubber.setDefaultProjection(Rubber_projection)
    Rubber = Rubber.reduceResolution(reducer = ee.Reducer.count(), bestEffort = True, maxPixels = 3*3).reproject(Hansen_projection)
    Rubber = Rubber.updateMask(Rubber.gt(4))       # Reduce resolution based on majority interpolation 
    Rubber = Rubber.where(Rubber.gt(0), Rubber_reclass_values)
    Rubber = Rubber.where(ManagedForests.gt(0).And(Rubber.gt(0)), Rubber.multiply(-1))
    """
    
    
    ## Analyze forest fire
    Forest_fire = Forest_fire.updateMask(TC_mask)
    Forest_fire = Forest_fire.updateMask(Forest_fire.gt(1))
    Forest_fire = Forest_fire.where(ManagedForests.gt(0).And(Forest_fire.gt(1)), Forest_fire.multiply(-1))
    
    
    
    ## Analyze dominant forest loss driver
    #dominant_driver = dominant_driver.clip(geometry)#$$
    dominant_driver = dominant_driver.updateMask(TC_mask)
    dominant_driver = dominant_driver.updateMask(dominant_driver.gt(0))
    # Convert dominant_driver to negative where it does not overlap with ForestManagement
    dominant_driver = dominant_driver.where(ManagedForests.gt(0).And(dominant_driver.gt(1)), dominant_driver.multiply(-1))

    
    
    
    ####################
    # 5. Analyse attribution (priority-based)
    ####################
    
    # Convert Hansen_loss to Hansen_loss_attribution
    Hansen_loss_attribution = Hansen_loss.rename('classification')
    
    # Attributing forest loss to OilPalm in Indonesia
    if Admin_boundary == 'Indonesia':
        # Since we are not considering Oil Palm values in Du et al plantation for Indonesia
        FL_att_to_old_plantations_species = FL_att_to_old_plantations_species.updateMask(FL_att_to_old_plantations_species.neq(Oilpalm_Plantation_reclass_value))
        FL_att_to_new_plantations_species = FL_att_to_new_plantations_species.where(FL_att_to_new_plantations_species.eq(Oilpalm_Plantation_reclass_value), 1)
    
        # Oil Palm Indonesia data is preferred over MapBiomas (directly converted in the reclassed function) or Forest plantation, thus all Oil Palm-MapBiomas pixels are converted to 1 (to include them in attribution pool)
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(OilPalm_Indonesia_Image.gt(0)), Hansen_loss_attribution.multiply(Oilpalm_Indonesia_reclass_value))
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(OilPalm_Indonesia_pre2000_Image.gt(0)), Hansen_loss_attribution.multiply(-Oilpalm_Indonesia_reclass_value))
    
    # Attributing forest loss to Maize-China
    if Admin_boundary == 'China':
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(20)).And(Maize_China_four), Maize_China_four)

    # Attributing forest loss to Soybean
    if Admin_boundary in MapBiomas_dataset_version_country:
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(22)).And(Soy_four.gt(0)), Soy_four)

    # Attributing forest loss to Sugarcane in Brazil
    if Admin_boundary == 'Brazil':
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(19)).And(Sugarcane_Brazil), Sugarcane_Brazil)
    
    # Attributing forest loss to OilPalm-Malaysia
    if Admin_boundary == 'Malaysia':
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(OilPalm_Malaysia_four), OilPalm_Malaysia_four)

    # Attributing forest loss to Cocoa
    if Admin_boundary in ['Côte d\'Ivoire', 'Ghana']: 
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(21)).And(Cocoa_map_CDG), Cocoa_map_CDG)

    # Attributing MapBiomas 'commodities' specifically (Match with commodities in the Supplementary data)
    if Admin_boundary in MapBiomas_dataset_version_country:
        if Admin_boundary == 'Bolivia':
            MapBiomas_endyear = 21 #For 2021
        else:
            MapBiomas_endyear = 22 #For 2022
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(MapBiomas_endyear)).And(MapBiomas.gte(3221)).And(MapBiomas.lte(3999)), MapBiomas) \
                                                         .where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(MapBiomas_endyear)).And(MapBiomas.eq(4000)), MapBiomas) \
                                                         .where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(MapBiomas_endyear)).And(MapBiomas.gte(6001)).And(MapBiomas.lte(6999)), MapBiomas)
    
    # Attributing forest loss to Rice
    if Admin_boundary in Rice_dataset_version_country:
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(19)).And(Paddy_rice), Paddy_rice)

    # Attributing forest loss to Rapeseed
    if Admin_boundary in Rapeseed_dataset_version_country:
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(19)).And(Rapeseed), Rapeseed)
    
    # Attributing forest loss to Rubber
    #$$Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(22)).And(Rubber), Rubber)

    if Admin_boundary not in Global_oilpalm_and_Coconut_not_in:
        # Attributing forest loss to OilPalm (Global; except Indonesia)
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(19)).And(OilPalm_Global), OilPalm_Global)
        
        # Attributing forest loss to Coconut
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(20)).And(Coconut), Coconut)

    # Attributing forest loss to Plantation species (old and new)
    if Admin_boundary in Plantation_dataset_version_country:
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(FL_att_to_old_plantations_species.gt(0)), FL_att_to_old_plantations_species.multiply(-1))
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(FL_att_to_new_plantations_species.gt(0)), FL_att_to_new_plantations_species)
        
    # Attributing forest loss to MapBiomas landuse
    if Admin_boundary in MapBiomas_dataset_version_country:
        Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(MapBiomas_endyear)).And(MapBiomas), MapBiomas)

    # Attributing forest loss to arable/temporary cropland
    Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Hansen_lossyear.lte(19)).And(combinedCropland), combinedCropland)

    # Attributing forest loss to Forest fire
    Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(Forest_fire), Forest_fire)

    # Attributing forest loss to Dominant drivers
    Hansen_loss_attribution = Hansen_loss_attribution.where(Hansen_loss_attribution.eq(1).And(dominant_driver), dominant_driver)

    
    
    ## Miscellaneous, since it is difficult to save files with special characters
    if Admin_boundary == 'México':
        Admin_boundary = 'Mexico'
    if Admin_boundary == 'Saint-Barthélemy':
        Admin_boundary = 'Saint-Barthelemy'
    if Admin_boundary =='Côte d\'Ivoire':
        Admin_boundary = 'Cote dIvoire'
    if Admin_boundary == 'Réunion':
        Admin_boundary = 'Reunion'
    if Admin_boundary == 'Åland':
        Admin_boundary = 'Aland'
    if Admin_boundary == 'São Tomé and Príncipe':
        Admin_boundary = 'Sao Tome and Principe'
    if Admin_boundary == 'Curaçao':
        Admin_boundary = 'Curacao'
    
    
    ####################
    # 6. Analyse above and below ground biomass
    ####################

    if simulation in ['All', 'AGB', 'BGB']:
        AGB = AGB.updateMask(TC_mask)
        #$$AGB = AGB.clip(geometry)
        AGB = AGB.multiply(44/12).multiply(0.47)  # Convert AGB to C, and then to CO2
        
        # Ecoregion for BGB calculation
        ecoRegions = ee.FeatureCollection(Ecoregion_dataset_version)
        ecoregionImage = ecoRegions.reduceToImage(
            properties=['BIOME_NUM'], 
            reducer=ee.Reducer.sum(),
        )
        
        ecoregionImage = ecoregionImage.updateMask(TC_mask)
        #$$ecoregionImage = ecoregionImage.clip(geometry)
        
        ## Analyse deadwood and litter
        elevation = elevation.updateMask(TC_mask)
        precipitation = precipitation.updateMask(TC_mask)
            
        # Define conditions
        tropical = ecoregionImage.eq(1).Or(ecoregionImage.eq(2)).Or(ecoregionImage.eq(3)).Or(ecoregionImage.eq(7))
        lowElevation = elevation.lt(2000)
        mediumPrecipitation = precipitation.gt(1000).And(precipitation.lte(1600))
        highPrecipitation = precipitation.gt(1600)
        highElevation = elevation.gt(2000)
        temperateBoreal = tropical.Not()
            
        # Apply conditions
        condition1 = tropical.And(lowElevation).And(precipitation.lt(1000))
        DeadWood_scale1 = ee.Image.constant(0.02)
        Litter_scale1 = ee.Image.constant(0.04)
            
        condition2 = tropical.And(lowElevation).And(mediumPrecipitation)
        DeadWood_scale2 = ee.Image.constant(0.01)
        Litter_scale2 = ee.Image.constant(0.01)
            
        condition3 = tropical.And(lowElevation).And(highPrecipitation)
        DeadWood_scale3 = ee.Image.constant(0.06)
        Litter_scale3 = ee.Image.constant(0.01)
            
        condition4 = tropical.And(highElevation)
        DeadWood_scale4 = ee.Image.constant(0.07)
        Litter_scale4 = ee.Image.constant(0.01)
            
        condition5 = temperateBoreal
        DeadWood_scale5 = ee.Image.constant(0.08)
        Litter_scale5 = ee.Image.constant(0.04)
            
        Deadwood = AGB.where(condition1, AGB.multiply(DeadWood_scale1)) \
                        .where(condition2, AGB.multiply(DeadWood_scale2)) \
                        .where(condition3, AGB.multiply(DeadWood_scale3)) \
                        .where(condition4, AGB.multiply(DeadWood_scale4)) \
                        .where(condition5, AGB.multiply(DeadWood_scale5))
            
        Litter = AGB.where(condition1, AGB.multiply(Litter_scale1)) \
                    .where(condition2, AGB.multiply(Litter_scale2)) \
                    .where(condition3, AGB.multiply(Litter_scale3)) \
                    .where(condition4, AGB.multiply(Litter_scale4)) \
                    .where(condition5, AGB.multiply(Litter_scale5))
        
        AGB_total = AGB.add(Deadwood).add(Litter)
    
        if simulation in ['All', 'BGB']:       
            # Define the BGB/AGB multiplication factors based on ecoregion value and AGB (Scale values define thje ration converted from Mg/ha to Mg/px)
            Root = ee.Image("projects/lu-chandrakant/assets/Root_Shoot_ratio/AROOT");
            Shoot = ee.Image("projects/lu-chandrakant/assets/Root_Shoot_ratio/ASHOOT");
                
            BGB_AGB = Root.divide(Shoot).unmask(0);
            
            # Apply conditional statements using ee.Image.where()
            BGB = AGB.multiply(BGB_AGB);


    if simulation in ['All', 'PEATLAND', 'SOC 0-30', 'SOC 30-100']:       
        # Clip the image to a specified geometry and apply a mask (TC_mask)
        Peatland = Peatland.updateMask(TC_mask)
        # Adding values from Hansen_loss_attribution to Peatland pixels
        Peatland = Hansen_loss_attribution.updateMask(Peatland) 
        
    
    ## Analyse Soil Organic Carbon Stock
    if simulation in ['All', 'SOC 0-30', 'SOC 30-100']:   
        # Calculate SOC stock for different depth ranges
        pixelarea = ee.Image.pixelArea().reproject(Hansen_projection).divide(10000)

    if simulation in ['All', 'SOC 0-30']:   
        # Combine SOC layers for different depth ranges
        SOC_0_30 = SOC_0_30.updateMask(TC_mask)
        # Converting tC/ha to tC/px
        SOC_0_30 = SOC_0_30.where(SOC_0_30, SOC_0_30.multiply(pixelarea))
        SOC_0_30 = SOC_0_30.multiply(44/12)  # Convert tC/px to tCO2/px
        SOC_0_30 = SOC_0_30.where(Peatland, SOC_0_30.multiply(0))


    if simulation in ['All', 'SOC 30-100']:   
        SOC_30_60_t_per_ha = calculate_SOC(30, 60)
        SOC_60_100_t_per_ha = calculate_SOC(60, 100)
        SOC_30_100 = (SOC_30_60_t_per_ha.add(SOC_60_100_t_per_ha)).updateMask(TC_mask)    
        SOC_30_100 = SOC_30_100.where(SOC_30_100, SOC_30_100.multiply(pixelarea))
        SOC_30_100 = SOC_30_100.multiply(44/12)    
        SOC_30_100 = SOC_30_100.where(Peatland, SOC_30_100.multiply(0))
        
    
        
    
    ####################
    # 7. Exporting attribution as csv
    ####################
    
    ## Export classified forest loss attribution as csv
    input = Hansen_loss_attribution

    def Classification_to_region_features(region):
        def to_class_feature(class_group):
            class_group = ee.Feature(class_group)
            year_groups = ee.List(class_group.get('groups'))
            areas = ee.Array(
                year_groups.map(
                    lambda properties: ee.Dictionary(properties).toArray().toList()
                )
            )
            labels = (
                areas.slice(1, 0, 1)
                .project([0])
                .toList()
                .map(
                    lambda year: ee.Number(year).toInt().add(2000).format('loss_%d'))
            )
            values = areas.slice(1, 1).project([0]).toList()
            return (
                ee.Feature(
                    None,
                    region.toDictionary(['CONTINENT', 'COUNTRY', 'GID_0', 'GID_1', 'GID_2'])
                )
                .set('Class', class_group.get('Class'))
                .set(ee.Dictionary.fromLists(labels, values))
            )

        groups = (
            ee.FeatureCollection(
                ee.List(
                    ee.Image.pixelArea().reproject(Hansen_projection).divide(1e4)
                    .addBands(Hansen_lossyear)
                    .addBands(input)
                    .reduceRegion(
                        reducer=ee.Reducer.sum().group(1, 'lossYear').group(2, 'Class'),
                        geometry=region.geometry(),
                        scale=Hansen_lossyear.projection().nominalScale(),
                        maxPixels=1e13,
                        tileScale=tileScale_value,
                    ).get('groups')
                ).map(
                    # Include a size, to filter out cases where no data exist
                    lambda group: ee.Feature(
                        None,
                        ee.Dictionary(group)
                    ).set(
                        'size',
                        ee.List(ee.Dictionary(group).get('groups')).size()
                    )
                )
            )
            .filter(ee.Filter.gt('size', 0))
        )
        return ee.FeatureCollection(groups.map(to_class_feature))
    
    
    
    ## Export AGB for each forest loss class as csv
    def AGB_Emission_to_region_features(region):
        def to_class_feature(class_group):
            class_group = ee.Feature(class_group)
            year_groups = ee.List(class_group.get('groups'))
            areas = ee.Array(
                year_groups.map(
                    lambda properties: ee.Dictionary(properties).toArray().toList()
                )
            )
            labels = (
                areas.slice(1, 0, 1)
                .project([0])
                .toList()
                .map(
                    lambda year: ee.Number(year).toInt().add(2000).format('loss_%d'))
            )
            values = areas.slice(1, 1).project([0]).toList()
            return (
                ee.Feature(
                    None,
                    region.toDictionary(['CONTINENT', 'COUNTRY', 'GID_0', 'GID_1', 'GID_2'])
                )
                .set('Class', class_group.get('Class'))
                .set(ee.Dictionary.fromLists(labels, values))
            )

        groups = (
            ee.FeatureCollection(
                ee.List(
                    AGB_total          # AGB is called here (includes Deadwood and Litter)
                    .addBands(Hansen_lossyear)
                    .addBands(input)
                    .reduceRegion(
                        reducer=ee.Reducer.sum().group(1, 'lossYear').group(2, 'Class'),
                        geometry=region.geometry(),
                        scale=Hansen_lossyear.projection().nominalScale(),
                        maxPixels=1e13,
                        tileScale=tileScale_value,
                    ).get('groups')
                ).map(
                    # Include a size, to filter out cases where no data exist
                    lambda group: ee.Feature(
                        None,
                        ee.Dictionary(group)
                    ).set(
                        'size',
                        ee.List(ee.Dictionary(group).get('groups')).size()
                    )
                )
            )
            .filter(ee.Filter.gt('size', 0))
        )
        return ee.FeatureCollection(groups.map(to_class_feature))
    
    
    
    ## Export BGB for each forest loss class as csv
    def BGB_Emission_to_region_features(region):
        def to_class_feature(class_group):
            class_group = ee.Feature(class_group)
            year_groups = ee.List(class_group.get('groups'))
            areas = ee.Array(
                year_groups.map(
                    lambda properties: ee.Dictionary(properties).toArray().toList()
                )
            )
            labels = (
                areas.slice(1, 0, 1)
                .project([0])
                .toList()
                .map(
                    lambda year: ee.Number(year).toInt().add(2000).format('loss_%d'))
            )
            values = areas.slice(1, 1).project([0]).toList()
            return (
                ee.Feature(
                    None,
                    region.toDictionary(['CONTINENT', 'COUNTRY', 'GID_0', 'GID_1', 'GID_2'])
                )
                .set('Class', class_group.get('Class'))
                .set(ee.Dictionary.fromLists(labels, values))
            )

        groups = (
            ee.FeatureCollection(
                ee.List(
                    BGB          # BGB is called here
                    .addBands(Hansen_lossyear)
                    .addBands(input)
                    .reduceRegion(
                        reducer=ee.Reducer.sum().group(1, 'lossYear').group(2, 'Class'),
                        geometry=region.geometry(),
                        scale=Hansen_lossyear.projection().nominalScale(),
                        maxPixels=1e13,
                        tileScale=tileScale_value,
                    ).get('groups')
                ).map(
                    # Include a size, to filter out cases where no data exist
                    lambda group: ee.Feature(
                        None,
                        ee.Dictionary(group)
                    ).set(
                        'size',
                        ee.List(ee.Dictionary(group).get('groups')).size()
                    )
                )
            )
            .filter(ee.Filter.gt('size', 0))
        )
        return ee.FeatureCollection(groups.map(to_class_feature))


    ## Export SOC 0-30m for each forest loss class as csv
    def SOC_0_30_Emission_to_region_features(region):
        def to_class_feature(class_group):
            class_group = ee.Feature(class_group)
            year_groups = ee.List(class_group.get('groups'))
            areas = ee.Array(
                year_groups.map(
                    lambda properties: ee.Dictionary(properties).toArray().toList()
                )
            )
            labels = (
                areas.slice(1, 0, 1)
                .project([0])
                .toList()
                .map(
                    lambda year: ee.Number(year).toInt().add(2000).format('loss_%d'))
            )
            values = areas.slice(1, 1).project([0]).toList()
            return (
                ee.Feature(
                    None,
                    region.toDictionary(['CONTINENT', 'COUNTRY', 'GID_0', 'GID_1', 'GID_2'])
                )
                .set('Class', class_group.get('Class'))
                .set(ee.Dictionary.fromLists(labels, values))
            )

        groups = (
            ee.FeatureCollection(
                ee.List(
                    SOC_0_30          # SOC 0-30 m
                    .addBands(Hansen_lossyear)
                    .addBands(input)
                    .reduceRegion(
                        reducer=ee.Reducer.sum().group(1, 'lossYear').group(2, 'Class'),
                        geometry=region.geometry(),
                        scale=Hansen_lossyear.projection().nominalScale(),
                        maxPixels=1e13,
                        tileScale=tileScale_value,
                    ).get('groups')
                ).map(
                    # Include a size, to filter out cases where no data exist
                    lambda group: ee.Feature(
                        None,
                        ee.Dictionary(group)
                    ).set(
                        'size',
                        ee.List(ee.Dictionary(group).get('groups')).size()
                    )
                )
            )
            .filter(ee.Filter.gt('size', 0))
        )
        return ee.FeatureCollection(groups.map(to_class_feature))


    ## Export SOC 30-100m for each forest loss class as csv
    def SOC_30_100_Emission_to_region_features(region):
        def to_class_feature(class_group):
            class_group = ee.Feature(class_group)
            year_groups = ee.List(class_group.get('groups'))
            areas = ee.Array(
                year_groups.map(
                    lambda properties: ee.Dictionary(properties).toArray().toList()
                )
            )
            labels = (
                areas.slice(1, 0, 1)
                .project([0])
                .toList()
                .map(
                    lambda year: ee.Number(year).toInt().add(2000).format('loss_%d'))
            )
            values = areas.slice(1, 1).project([0]).toList()
            return (
                ee.Feature(
                    None,
                    region.toDictionary(['CONTINENT', 'COUNTRY', 'GID_0', 'GID_1', 'GID_2'])
                )
                .set('Class', class_group.get('Class'))
                .set(ee.Dictionary.fromLists(labels, values))
            )

        groups = (
            ee.FeatureCollection(
                ee.List(
                    SOC_30_100          # SOC 30-100 m
                    .addBands(Hansen_lossyear)
                    .addBands(input)
                    .reduceRegion(
                        reducer=ee.Reducer.sum().group(1, 'lossYear').group(2, 'Class'),
                        geometry=region.geometry(),
                        scale=Hansen_lossyear.projection().nominalScale(),
                        maxPixels=1e13,
                        tileScale=tileScale_value,
                    ).get('groups')
                ).map(
                    # Include a size, to filter out cases where no data exist
                    lambda group: ee.Feature(
                        None,
                        ee.Dictionary(group)
                    ).set(
                        'size',
                        ee.List(ee.Dictionary(group).get('groups')).size()
                    )
                )
            )
            .filter(ee.Filter.gt('size', 0))
        )
        return ee.FeatureCollection(groups.map(to_class_feature))


    def Classification_to_Peatland_regions(region):
        def to_class_feature(class_group):
            class_group = ee.Feature(class_group)
            year_groups = ee.List(class_group.get('groups'))
            areas = ee.Array(
                year_groups.map(
                    lambda properties: ee.Dictionary(properties).toArray().toList()
                )
            )
            labels = (
                areas.slice(1, 0, 1)
                .project([0])
                .toList()
                .map(
                    lambda year: ee.Number(year).toInt().add(2000).format('loss_%d'))
            )
            values = areas.slice(1, 1).project([0]).toList()
            return (
                ee.Feature(
                    None,
                    region.toDictionary(['CONTINENT', 'COUNTRY', 'GID_0', 'GID_1', 'GID_2'])
                )
                .set('Class', class_group.get('Class'))
                .set(ee.Dictionary.fromLists(labels, values))
            )

        groups = (
            ee.FeatureCollection(
                ee.List(
                    ee.Image.pixelArea().reproject(Hansen_projection).divide(1e4)
                    .addBands(Hansen_lossyear)
                    .addBands(Peatland)                 #Changed data to Peatland for attribution
                    .reduceRegion(
                        reducer=ee.Reducer.sum().group(1, 'lossYear').group(2, 'Class'),
                        geometry=region.geometry(),
                        scale=Hansen_lossyear.projection().nominalScale(),
                        maxPixels=1e13,
                        tileScale=tileScale_value,
                    ).get('groups')
                ).map(
                    # Include a size, to filter out cases where no data exist
                    lambda group: ee.Feature(
                        None,
                        ee.Dictionary(group)
                    ).set(
                        'size',
                        ee.List(ee.Dictionary(group).get('groups')).size()
                    )
                )
            )
            .filter(ee.Filter.gt('size', 0))
        )
        return ee.FeatureCollection(groups.map(to_class_feature))
    
    
    # Only exports classification
    def Forest_loss_attribution(region, LOSS, runID = ''):
        if LOSS == 'CLASSIFICATION':
            areas = region.map(Classification_to_region_features).flatten()
            savefilename = 'CLASSIFICATION'
        elif LOSS == 'PEATLAND':
            areas = region.map(Classification_to_Peatland_regions).flatten()
            savefilename = 'PEATLAND'
        propertiesToExport = ['CONTINENT', 'COUNTRY', 'GID_0', 'GID_1', 'GID_2', 'Class']
        for i in range(2001, end_year+1):
            propertiesToExport.append('loss_' + str(i))

        task = ee.batch.Export.table.toDrive(
            collection=areas,
            description= 'Forest_loss_to_'+savefilename+'_'+ str(Admin_boundary)+'_'+ runID + '_' +'PythonAPI_'+ str(datetime.datetime.now().isoformat()[0:19]),
            folder='DeDuCE_v'+str(GEE_version)+sub_folder_code,
            fileFormat='CSV',
            selectors=propertiesToExport
        )
        task.start()
        
    # Exports AGB and BGB
    def Forest_loss_emission_attribution(region, emission, runID = ''):
        if emission == 'BGB':
            areas = region.map(BGB_Emission_to_region_features).flatten()
            savefilename = 'BGB_EMISSION'
        elif emission == 'SOC 0-30':
            areas = region.map(SOC_0_30_Emission_to_region_features).flatten()
            savefilename = 'SOC_0_30'
        elif emission == 'SOC 30-100':
            areas = region.map(SOC_30_100_Emission_to_region_features).flatten()
            savefilename = 'SOC_30_100'
        elif emission == 'AGB':
            areas = region.map(AGB_Emission_to_region_features).flatten()
            savefilename = 'AGB_EMISSION'
        propertiesToExport = ['CONTINENT', 'COUNTRY', 'GID_0', 'GID_1', 'GID_2', 'Class']
        for i in range(2001, end_year+1):
            propertiesToExport.append('loss_' + str(i))

        task = ee.batch.Export.table.toDrive(
            collection=areas,
            description= 'Forest_loss_to_'+savefilename+'_'+ str(Admin_boundary)+'_'+ runID + '_' +'PythonAPI_'+ str(datetime.datetime.now().isoformat()[0:19]),
            folder='DeDuCE_v'+str(GEE_version)+sub_folder_code,
            fileFormat='CSV',
            selectors=propertiesToExport
        )
        task.start()
    
    
    ####################
    # Funtions that run the export
    ####################
    tileScale_value = 2

    if simulation == 'CLASSIFICATION':
        Forest_loss_attribution(geometry, 'CLASSIFICATION')
    elif simulation == 'PEATLAND':
        Forest_loss_attribution(geometry, 'PEATLAND')
    elif simulation == 'AGB':
        Forest_loss_emission_attribution(geometry, 'AGB')
    elif simulation == 'BGB':
        Forest_loss_emission_attribution(geometry, 'BGB')
    elif simulation == 'SOC 0-30':
        Forest_loss_emission_attribution(geometry, 'SOC 0-30')
    elif simulation == 'SOC 30-100':
        Forest_loss_emission_attribution(geometry, 'SOC 30-100')
    else:
        Forest_loss_attribution(geometry, 'CLASSIFICATION')
        Forest_loss_attribution(geometry, 'PEATLAND')
        Forest_loss_emission_attribution(geometry, 'AGB')
        Forest_loss_emission_attribution(geometry, 'BGB')
        Forest_loss_emission_attribution(geometry, 'SOC 0-30')
        Forest_loss_emission_attribution(geometry, 'SOC 30-100')


In [6]:
GEE_loop('Uruguay')

Boundary Uruguay : 141


In [ ]:
import time
for Country in ['United States']:
    for simulation in ['CLASSIFICATION','PEATLAND','AGB','BGB','SOC 0-30','SOC 30-100']:
        print(Country, simulation)
        # Load the feature collection needed to simulate aggregation
        while True:
            if all(task.status()['state'] not in ['RUNNING', 'READY'] for task in ee.batch.Task.list()[:5]):
                GEE_loop(Country, simulation)
                break
            else:
                #print('Waiting')
                time.sleep(20)  # Waits 2 minutes before re-checking, as suggested

Japan CLASSIFICATION


In [ ]:
['Romania', #Tile scale = 4; GID_0; SOC's
 'Japan','México','Thailand','Algeria','Colombia', 'Vietnam','Philippines', #Tile scale = 2; GID_0
 'United States','Russia','Canada','Brazil'] #Tile scale = 2; GID_1